### Setup

### Fine-tuning

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


In [ ]:
import datasets

[knowrohit07/know_medical_dialogue_v2](https://huggingface.co/datasets/knowrohit07/know_medical_dialogue_v2)

In [ ]:
from datasets import load_dataset

ds = load_dataset("knowrohit07/know_medical_dialogue_v2")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 6307
    })
})

In [ ]:
trn = ds['train']
trn[3]

{'output': 'again, really need complete work up and medical notes to provide any insight here',
 'input': '',
 'instruction': "Appreciate it al labs have come back completely negative only thing that's been consistently elevated but wasn't that much of a concern was CK levels everything else you could think of from any type of test normal"}

`accelerate launch -m axolotl.cli.train sql.yml`

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

{'output': 'again, really need complete work up and medical notes to provide any insight here',
 'input': '',
 'instruction': "Appreciate it al labs have come back completely negative only thing that's been consistently elevated but wasn't that much of a concern was CK levels everything else you could think of from any type of test normal",
 'question': 'Get the count of competition hosts by theme.'}

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])

# TEST

In [ ]:
# Install the necessary packages
!pip install transformers datasets torch
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
!pip install transformers datasets torch

In [ ]:
# Loading the dataset
dataset = load_dataset('emotion')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Tokenizing the dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U

In [ ]:
# Setting up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
10,1.894000
20,1.837300
30,1.789100
40,1.780000
50,1.703200
60,1.762300
70,1.699300
80,1.694100
90,1.621000
100,1.550000


TrainOutput(global_step=3000, training_loss=0.28815643274784086, metrics={'train_runtime': 1479.2606, 'train_samples_per_second': 32.449, 'train_steps_per_second': 2.028, 'total_flos': 1.2629784231936e+16, 'train_loss': 0.28815643274784086, 'epoch': 3.0})

In [ ]:
# Evaluate the model
trainer.evaluate(tokenized_datasets['test'])

{'eval_loss': 0.15972067415714264,
 'eval_runtime': 19.515,
 'eval_samples_per_second': 102.485,
 'eval_steps_per_second': 1.64,
 'epoch': 3.0}

# Conclusion
In this portion of the assignment, we have successfully downloaded a model and fine-tuned it to a test dataset. In this case, the dataset was an emotion dataset.